In [7]:
import pandas as pd
import numpy as np
import streamlit as st

In [8]:
brasileirao = pd.read_excel('../data/brasileirao2023.xlsx')
brasileirao

,Temporada,Rodada,Mandante,Score_m,Visitante,Score_v
0,2023,1,PAL,2.0,CUI,1.0
1,2023,1,AME,0.0,FLU,3.0
2,2023,1,CAP,2.0,GOI,0.0
3,2023,1,BOT,2.0,SAO,1.0
4,2023,1,RED,2.0,BAH,1.0
...,...,...,...,...,...,...
375,2023,38,GOI,NaN,AME,NaN
376,2023,38,SAN,NaN,FOR,NaN
377,2023,38,SAO,NaN,FLA,NaN
378,2023,38,VAS,NaN,RED,NaN


In [9]:
brasileirao['Score_m']

0      2.0
1      0.0
2      2.0
3      2.0
4      2.0
      ... 
375    NaN
376    NaN
377    NaN
378    NaN
379    NaN
Name: Score_m, Length: 380, dtype: float64

In [10]:
brasileirao[brasileirao['Score_m'].notnull()]['Rodada'].count()

280

In [45]:
brasileirao = brasileirao[brasileirao['Score_m'].notnull()]
brasileirao

,Temporada,Rodada,Mandante,Score_m,Visitante,Score_v
0,2023,1,PAL,2.0,CUI,1.0
1,2023,1,AME,0.0,FLU,3.0
2,2023,1,CAP,2.0,GOI,0.0
3,2023,1,BOT,2.0,SAO,1.0
4,2023,1,RED,2.0,BAH,1.0
...,...,...,...,...,...,...
275,2023,28,CAM,0.0,CRU,1.0
276,2023,28,COR,1.0,AME,1.0
277,2023,28,SAO,3.0,GRE,0.0
278,2023,28,BOT,1.0,CAP,1.0


In [44]:
brasileirao.dtypes

Temporada      int64
Rodada         int64
Mandante      object
Score_m      float64
Visitante     object
Score_v      float64
dtype: object

In [12]:
times = brasileirao['Mandante'].unique()
tabela = pd.DataFrame()
for time in times:
    vit = emp = der = pro = con = jog = 0
    # emp = 0
    # der = 0
    # pro = 0
    # con = 0
    # jog = 0
    mandante = brasileirao[brasileirao['Mandante'] == time]
    for indice, partida in mandante.iterrows():
        if partida['Score_m'] > partida['Score_v']:
            vit += 1 
        elif partida['Score_m'] == partida['Score_v']:
            emp += 1
        else:
            der += 1
        pro += partida['Score_m']
        con += partida['Score_v']
        jog += 1
    visitante = brasileirao[brasileirao['Visitante'] == time]
    for indice, partida in visitante.iterrows():
        if partida['Score_v'] > partida['Score_m']:
            vit += 1
        elif partida['Score_v'] == partida['Score_m']:
            emp += 1
        else:
            der += 1
        pro += partida['Score_v']
        con += partida['Score_m']
        jog += 1
    new_row = pd.DataFrame({
        '1-Time': [time],
        '3-J' : [jog],
        '4-V' : [vit],
        '5-E' : [emp],
        '6-D' : [der],
        '7-GP': [pro],
        '8-GC': [con]
    })   
    tabela = pd.concat([tabela, new_row], ignore_index = True)
    
tabela['2-P'] = (tabela['4-V'] * 3) + tabela['5-E']
tabela['9-SG'] = tabela['7-GP'] - tabela['8-GC']   
tabela_sort = tabela.sort_values(by=['2-P','4-V','9-SG'], ascending=False)
tabela_sort

,1-Time,3-J,4-V,5-E,6-D,7-GP,8-GC,2-P,9-SG
3,BOT,28,18,5,5,45.0,18.0,59,27.0
4,RED,28,14,10,4,40.0,24.0,52,16.0
8,FLA,28,14,8,6,41.0,31.0,50,10.0
0,PAL,28,13,8,7,42.0,24.0,47,18.0
2,CAP,28,12,9,7,41.0,32.0,45,9.0
9,GRE,28,13,5,10,43.0,40.0,44,3.0
6,CAM,28,12,7,9,32.0,22.0,43,10.0
5,FOR,28,12,6,10,33.0,29.0,42,4.0
10,FLU,28,12,6,10,37.0,35.0,42,2.0
12,SAO,28,10,8,10,34.0,29.0,38,5.0


In [13]:
tabela_sort[tabela_sort['7-GP'].notnull().sort_values(ascending=False)]['1-Time'].iloc[0]

C:\Users\samue\AppData\Local\Temp\ipykernel_17116\1631230441.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tabela_sort[tabela_sort['7-GP'].notnull().sort_values(ascending=False)]['1-Time'].iloc[0]


'BOT'

In [14]:
a = brasileirao[brasileirao['Score_m'].notnull()]['Rodada'].unique().max()
a

28

In [15]:
from sklearn.cluster import KMeans
df_data = tabela[[
    '2-P',              
    '4-V',
    '5-E',
    '6-D',
    '9-SG'
]]
kmeans = KMeans(n_clusters=5, random_state=0).fit(df_data)
tabela['cluster'] = kmeans.labels_
# tabela['c_p'] = kmeans.cluster_centers_[kmeans.labels_,0]
# tabela['c_v'] = kmeans.cluster_centers_[kmeans.labels_,1]
# tabela['c_e'] = kmeans.cluster_centers_[kmeans.labels_,2]
# tabela['c_d'] = kmeans.cluster_centers_[kmeans.labels_,3]
# tabela['c_sg'] = kmeans.cluster_centers_[kmeans.labels_,4]
tabela

c:\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,1-Time,3-J,4-V,5-E,6-D,7-GP,8-GC,2-P,9-SG,cluster
0,PAL,28,13,8,7,42.0,24.0,47,18.0,2
1,AME,28,4,7,17,31.0,57.0,19,-26.0,0
2,CAP,28,12,9,7,41.0,32.0,45,9.0,1
3,BOT,28,18,5,5,45.0,18.0,59,27.0,4
4,RED,28,14,10,4,40.0,24.0,52,16.0,2
5,FOR,28,12,6,10,33.0,29.0,42,4.0,1
6,CAM,28,12,7,9,32.0,22.0,43,10.0,1
7,COR,28,7,12,9,34.0,36.0,33,-2.0,3
8,FLA,28,14,8,6,41.0,31.0,50,10.0,2
9,GRE,28,13,5,10,43.0,40.0,44,3.0,1


In [16]:
print(kmeans.labels_)

[2 0 1 4 2 1 1 3 2 1 1 3 1 3 3 3 0 0 3 3]


In [17]:
kmeans.cluster_centers_

array([[ 23.        ,   5.66666667,   6.        ,  16.33333333,
        -25.66666667],
       [ 42.33333333,  11.83333333,   6.83333333,   9.33333333,
          5.5       ],
       [ 49.66666667,  13.66666667,   8.66666667,   5.66666667,
         14.66666667],
       [ 33.42857143,   8.28571429,   8.57142857,  11.14285714,
         -3.85714286],
       [ 59.        ,  18.        ,   5.        ,   5.        ,
         27.        ]])

In [18]:
df_cluster = pd.DataFrame()
for cluster, colunas in enumerate(kmeans.cluster_centers_):
    print(cluster)
    print(colunas)
    new_row = pd.DataFrame({
        'cluster': [cluster], 
        '2-P': [colunas[0]],              
        '4-V': [colunas[1]],
        '5-E': [colunas[2]],
        '6-D': [colunas[3]],
        '9-SG': [colunas[4]],
    })
    df_cluster = pd.concat([df_cluster, new_row], ignore_index=True)
df_cluster

0
[ 23.           5.66666667   6.          16.33333333 -25.66666667]
1
[42.33333333 11.83333333  6.83333333  9.33333333  5.5       ]
2
[49.66666667 13.66666667  8.66666667  5.66666667 14.66666667]
3
[33.42857143  8.28571429  8.57142857 11.14285714 -3.85714286]
4
[59. 18.  5.  5. 27.]


,cluster,2-P,4-V,5-E,6-D,9-SG
0,0,23.000000,5.666667,6.000000,16.333333,-25.666667
1,1,42.333333,11.833333,6.833333,9.333333,5.500000
2,2,49.666667,13.666667,8.666667,5.666667,14.666667
3,3,33.428571,8.285714,8.571429,11.142857,-3.857143
4,4,59.000000,18.000000,5.000000,5.000000,27.000000


In [19]:
df_cluster_sort = df_cluster.sort_values(by='2-P', ascending=False)
df_cluster_sort

,cluster,2-P,4-V,5-E,6-D,9-SG
4,4,59.000000,18.000000,5.000000,5.000000,27.000000
2,2,49.666667,13.666667,8.666667,5.666667,14.666667
1,1,42.333333,11.833333,6.833333,9.333333,5.500000
3,3,33.428571,8.285714,8.571429,11.142857,-3.857143
0,0,23.000000,5.666667,6.000000,16.333333,-25.666667


In [20]:
df_cluster_sort['grupo'] = ['Título','Libertadores','Sul-Americana','Limbo','Rebaixamento']
df_cluster_sort

,cluster,2-P,4-V,5-E,6-D,9-SG,grupo
4,4,59.000000,18.000000,5.000000,5.000000,27.000000,Título
2,2,49.666667,13.666667,8.666667,5.666667,14.666667,Libertadores
1,1,42.333333,11.833333,6.833333,9.333333,5.500000,Sul-Americana
3,3,33.428571,8.285714,8.571429,11.142857,-3.857143,Limbo
0,0,23.000000,5.666667,6.000000,16.333333,-25.666667,Rebaixamento


In [21]:
df_cluster_grupo = df_cluster_sort[['cluster','grupo']]
df_cluster_grupo

,cluster,grupo
4,4,Título
2,2,Libertadores
1,1,Sul-Americana
3,3,Limbo
0,0,Rebaixamento


In [22]:
tabela

,1-Time,3-J,4-V,5-E,6-D,7-GP,8-GC,2-P,9-SG,cluster
0,PAL,28,13,8,7,42.0,24.0,47,18.0,2
1,AME,28,4,7,17,31.0,57.0,19,-26.0,0
2,CAP,28,12,9,7,41.0,32.0,45,9.0,1
3,BOT,28,18,5,5,45.0,18.0,59,27.0,4
4,RED,28,14,10,4,40.0,24.0,52,16.0,2
5,FOR,28,12,6,10,33.0,29.0,42,4.0,1
6,CAM,28,12,7,9,32.0,22.0,43,10.0,1
7,COR,28,7,12,9,34.0,36.0,33,-2.0,3
8,FLA,28,14,8,6,41.0,31.0,50,10.0,2
9,GRE,28,13,5,10,43.0,40.0,44,3.0,1


In [23]:
tabela_sort = tabela.sort_values(by=['2-P','4-V','9-SG'], ascending=False)
tabela_sort.style.background_gradient(cmap='Blues')
# tabela_sort.style.background_gradient(cmap='Blues').format("{:.2f}").render()


,1-Time,3-J,4-V,5-E,6-D,7-GP,8-GC,2-P,9-SG,cluster
3,BOT,28,18,5,5,45.000000,18.000000,59,27.000000,4
4,RED,28,14,10,4,40.000000,24.000000,52,16.000000,2
8,FLA,28,14,8,6,41.000000,31.000000,50,10.000000,2
0,PAL,28,13,8,7,42.000000,24.000000,47,18.000000,2
2,CAP,28,12,9,7,41.000000,32.000000,45,9.000000,1
9,GRE,28,13,5,10,43.000000,40.000000,44,3.000000,1
6,CAM,28,12,7,9,32.000000,22.000000,43,10.000000,1
5,FOR,28,12,6,10,33.000000,29.000000,42,4.000000,1
10,FLU,28,12,6,10,37.000000,35.000000,42,2.000000,1
12,SAO,28,10,8,10,34.000000,29.000000,38,5.000000,1


In [24]:
#NORMALIZAR A TABELA E COLORIR...

#grafico com pontuacao original e dos cluster

#grafico dos cluster

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler


In [25]:
df_data = tabela[[
    'cluster',              
    '2-P',              
    '4-V',
    '5-E',
    '6-D',
    '9-SG'
]]

In [26]:
X_Train = df_data.drop(columns=['cluster'], axis=1)
X_Test = df_data.drop(columns=['cluster'], axis=1)
y_Train = df_data['cluster']
y_Test = df_data['cluster']

In [27]:
sc_x = StandardScaler()
X_Train = sc_x.fit_transform(X_Train)
X_Test = sc_x.fit_transform(X_Test)

In [28]:
logreg = LogisticRegression(solver="lbfgs", max_iter=500)
logreg.fit(X_Train, y_Train)
pred_logreg = logreg.predict(X_Test)
pred_proba = logreg.predict_proba(X_Test)

In [29]:
tabela["cluster_pred"] = pred_logreg

In [30]:
lista_proba = pred_proba.tolist()

data = []  # Lista para armazenar os dicionários de dados

index = 0
for proba in lista_proba:
    for i in range(0, len(proba)):
        new_row = {"index": index, "prob": i, "valor": round(proba[i], 4)}
        data.append(new_row)
    index += 1

df_prob = pd.DataFrame(data)  # Cria o DataFrame a partir da lista de dicionários
df_prob = df_prob.pivot_table(
    index="index", columns="prob", values="valor", aggfunc="sum"
)
df_prob = df_prob.reset_index()
df_prob = df_prob.set_index("index")


In [31]:
df_prob = df_prob.rename(columns={
    0.0: 'cl_o',
    1.0: 'cl_1',
    2.0: 'cl_2',
    3.0: 'cl_3',
    4.0: 'cl_4'
})

In [32]:
tabela = pd.merge(tabela, df_prob, left_index=True, right_index=True)
tabela

,1-Time,3-J,4-V,5-E,6-D,7-GP,8-GC,2-P,9-SG,cluster,cluster_pred,cl_o,cl_1,cl_2,cl_3,cl_4
0,PAL,28,13,8,7,42.0,24.0,47,18.0,2,2,0.0011,0.4165,0.4207,0.0858,0.0759
1,AME,28,4,7,17,31.0,57.0,19,-26.0,0,0,0.7051,0.0060,0.0000,0.2889,0.0000
2,CAP,28,12,9,7,41.0,32.0,45,9.0,1,2,0.0031,0.3560,0.4091,0.2008,0.0310
3,BOT,28,18,5,5,45.0,18.0,59,27.0,4,4,0.0000,0.2315,0.2387,0.0011,0.5287
4,RED,28,14,10,4,40.0,24.0,52,16.0,2,2,0.0001,0.1294,0.8043,0.0287,0.0375
5,FOR,28,12,6,10,33.0,29.0,42,4.0,1,1,0.0245,0.6792,0.0783,0.1730,0.0451
6,CAM,28,12,7,9,32.0,22.0,43,10.0,1,1,0.0091,0.5954,0.1625,0.1828,0.0502
7,COR,28,7,12,9,34.0,36.0,33,-2.0,3,3,0.0114,0.0351,0.0378,0.9156,0.0002
8,FLA,28,14,8,6,41.0,31.0,50,10.0,2,2,0.0010,0.3808,0.4820,0.0522,0.0840
9,GRE,28,13,5,10,43.0,40.0,44,3.0,1,1,0.0201,0.7589,0.0617,0.0845,0.0749


In [33]:
df_pred = tabela[[
    '1-Time',
    'cl_o',
    'cl_1',
    'cl_2',
    'cl_3',
    'cl_4',
]]
df_pred = df_pred.copy()
df_pred

,1-Time,cl_o,cl_1,cl_2,cl_3,cl_4
0,PAL,0.0011,0.4165,0.4207,0.0858,0.0759
1,AME,0.7051,0.0060,0.0000,0.2889,0.0000
2,CAP,0.0031,0.3560,0.4091,0.2008,0.0310
3,BOT,0.0000,0.2315,0.2387,0.0011,0.5287
4,RED,0.0001,0.1294,0.8043,0.0287,0.0375
5,FOR,0.0245,0.6792,0.0783,0.1730,0.0451
6,CAM,0.0091,0.5954,0.1625,0.1828,0.0502
7,COR,0.0114,0.0351,0.0378,0.9156,0.0002
8,FLA,0.0010,0.3808,0.4820,0.0522,0.0840
9,GRE,0.0201,0.7589,0.0617,0.0845,0.0749


In [34]:
df_pred.style.background_gradient(cmap='Blues', axis=1)

,1-Time,cl_o,cl_1,cl_2,cl_3,cl_4
0,PAL,0.001100,0.416500,0.420700,0.085800,0.075900
1,AME,0.705100,0.006000,0.000000,0.288900,0.000000
2,CAP,0.003100,0.356000,0.409100,0.200800,0.031000
3,BOT,0.000000,0.231500,0.238700,0.001100,0.528700
4,RED,0.000100,0.129400,0.804300,0.028700,0.037500
5,FOR,0.024500,0.679200,0.078300,0.173000,0.045100
6,CAM,0.009100,0.595400,0.162500,0.182800,0.050200
7,COR,0.011400,0.035100,0.037800,0.915600,0.000200
8,FLA,0.001000,0.380800,0.482000,0.052200,0.084000
9,GRE,0.020100,0.758900,0.061700,0.084500,0.074900


In [35]:
rodadas = brasileirao['Rodada'].unique()
times = brasileirao['Mandante'].unique()
pontuacao = pd.DataFrame()
data = []  # Lista para armazenar os dicionários de dados

for rodada in rodadas:
    for time in times:
        resultado = brasileirao[(brasileirao['Rodada'] == rodada) & ((brasileirao['Mandante'] == time) | (brasileirao['Visitante'] == time))]
        if len(resultado) > 0:
            resultado = resultado.reset_index()
            if resultado['Mandante'][0] == time:
                if resultado['Score_m'][0] > resultado['Score_v'][0]:
                    pontos = 3
                elif resultado['Score_m'][0] == resultado['Score_v'][0]:
                    pontos = 1
                else:
                    pontos = 0
            else:                  
                if resultado['Score_v'][0] > resultado['Score_m'][0]:
                    pontos = 3
                elif resultado['Score_v'][0] == resultado['Score_m'][0]:
                    pontos = 1
                else:
                    pontos = 0
            new_row = {
                'time': time,
                'rodada': rodada,
                'pontos': pontos
            }        
            data.append(new_row)  # Adiciona o dicionário à lista de dados

pontuacao = pd.DataFrame(data)  # Cria o DataFrame a partir da lista de dicionários

pt_pontuacao = pontuacao.pivot_table(index='rodada', columns='time', values='pontos', aggfunc='sum')
pt_pontuacao_cum = pt_pontuacao.cumsum()
pt_pontuacao_cum

time,AME,BAH,BOT,CAM,CAP,CFC,COR,CRU,CUI,FLA,FLU,FOR,GOI,GRE,INT,PAL,RED,SAN,SAO,VAS
rodada,,,,,,,,,,,,,,,,,,,,
1,0,0,3,0,3,0,3,0,0,3,3,1,0,3,1,3,3,0,0,3
2,0,0,6,1,3,0,3,3,1,3,6,4,3,3,4,4,4,1,3,4
3,0,3,9,4,3,1,3,6,1,3,6,7,3,6,7,7,4,4,4,4
4,0,6,12,4,6,1,4,9,4,3,7,8,3,7,7,10,5,4,7,5
5,1,6,15,7,9,2,4,9,4,6,10,9,3,7,7,13,6,7,8,6
6,1,6,15,10,12,2,5,12,4,9,13,10,6,8,7,14,7,10,9,6
7,4,7,18,13,12,2,5,12,7,12,13,10,7,11,7,15,10,11,12,6
8,4,7,21,14,12,3,8,13,8,13,13,13,7,14,10,16,13,11,15,6
9,7,8,21,17,15,3,8,13,11,16,16,14,7,17,11,19,13,12,15,6


In [36]:
pt_pontuacao_cum = pt_pontuacao_cum.reset_index()
pt_pontuacao_cum

time,rodada,AME,BAH,BOT,CAM,CAP,CFC,COR,CRU,CUI,...,FLU,FOR,GOI,GRE,INT,PAL,RED,SAN,SAO,VAS
0,1,0,0,3,0,3,0,3,0,0,...,3,1,0,3,1,3,3,0,0,3
1,2,0,0,6,1,3,0,3,3,1,...,6,4,3,3,4,4,4,1,3,4
2,3,0,3,9,4,3,1,3,6,1,...,6,7,3,6,7,7,4,4,4,4
3,4,0,6,12,4,6,1,4,9,4,...,7,8,3,7,7,10,5,4,7,5
4,5,1,6,15,7,9,2,4,9,4,...,10,9,3,7,7,13,6,7,8,6
5,6,1,6,15,10,12,2,5,12,4,...,13,10,6,8,7,14,7,10,9,6
6,7,4,7,18,13,12,2,5,12,7,...,13,10,7,11,7,15,10,11,12,6
7,8,4,7,21,14,12,3,8,13,8,...,13,13,7,14,10,16,13,11,15,6
8,9,7,8,21,17,15,3,8,13,11,...,16,14,7,17,11,19,13,12,15,6
9,10,8,9,24,18,16,4,9,14,12,...,17,14,8,17,14,22,14,13,15,6


In [37]:
from sklearn.linear_model import LinearRegression

In [38]:
colunas = pt_pontuacao_cum.columns
df_regressao = pd.DataFrame()
data_regressao = []  # Lista para armazenar os dicionários de resultados de regressão

for coluna in colunas:
    if coluna != 'rodada':
        X = pt_pontuacao_cum['rodada'].values.reshape(-1, 1)
        y = pt_pontuacao_cum[coluna].values.reshape(-1, 1)
        regressor = LinearRegression()
        regressor.fit(X, y)
        A = regressor.intercept_[0]
        B = regressor.coef_[0][0]
        x = A + (B * 38)
        new_row_regressao = {
            'time': coluna,
            'intercept': round(A, 2),
            'slope': round(B, 2),
            'pontuacao_final': round(x, 2)
        }
        data_regressao.append(new_row_regressao)

df_regressao = pd.DataFrame(data_regressao)
df_regressao.sort_values(by='pontuacao_final', ascending=False)

,time,intercept,slope,pontuacao_final
2,BOT,4.08,2.08,83.15
16,RED,-2.23,1.83,67.17
9,FLA,-0.67,1.77,66.69
13,GRE,0.73,1.70,65.36
15,PAL,2.97,1.64,65.20
4,CAP,-0.03,1.60,60.61
10,FLU,2.02,1.54,60.37
3,CAM,-0.18,1.50,56.92
11,FOR,0.66,1.48,56.87
8,CUI,-2.48,1.42,51.65


In [39]:
df_regressao = df_regressao[['time','slope']]
df_regressao

,time,slope
0,AME,0.74
1,BAH,1.11
2,BOT,2.08
3,CAM,1.50
4,CAP,1.60
5,CFC,0.80
6,COR,1.26
7,CRU,1.10
8,CUI,1.42
9,FLA,1.77


In [40]:
tabela = tabela.merge(df_regressao, left_on='1-Time', right_on='time', how='left')
tabela

,1-Time,3-J,4-V,5-E,6-D,7-GP,8-GC,2-P,9-SG,cluster,cluster_pred,cl_o,cl_1,cl_2,cl_3,cl_4,time,slope
0,PAL,28,13,8,7,42.0,24.0,47,18.0,2,2,0.0011,0.4165,0.4207,0.0858,0.0759,PAL,1.64
1,AME,28,4,7,17,31.0,57.0,19,-26.0,0,0,0.7051,0.0060,0.0000,0.2889,0.0000,AME,0.74
2,CAP,28,12,9,7,41.0,32.0,45,9.0,1,2,0.0031,0.3560,0.4091,0.2008,0.0310,CAP,1.60
3,BOT,28,18,5,5,45.0,18.0,59,27.0,4,4,0.0000,0.2315,0.2387,0.0011,0.5287,BOT,2.08
4,RED,28,14,10,4,40.0,24.0,52,16.0,2,2,0.0001,0.1294,0.8043,0.0287,0.0375,RED,1.83
5,FOR,28,12,6,10,33.0,29.0,42,4.0,1,1,0.0245,0.6792,0.0783,0.1730,0.0451,FOR,1.48
6,CAM,28,12,7,9,32.0,22.0,43,10.0,1,1,0.0091,0.5954,0.1625,0.1828,0.0502,CAM,1.50
7,COR,28,7,12,9,34.0,36.0,33,-2.0,3,3,0.0114,0.0351,0.0378,0.9156,0.0002,COR,1.26
8,FLA,28,14,8,6,41.0,31.0,50,10.0,2,2,0.0010,0.3808,0.4820,0.0522,0.0840,FLA,1.77
9,GRE,28,13,5,10,43.0,40.0,44,3.0,1,1,0.0201,0.7589,0.0617,0.0845,0.0749,GRE,1.70


In [41]:
from sklearn.cluster import KMeans
df_data = tabela[[
    '2-P',              
    '4-V',
    '5-E',
    '6-D',
    '9-SG',
    'slope'
]]
kmeans = KMeans(n_clusters=5, random_state=0).fit(df_data)
tabela['cluster'] = kmeans.labels_
tabela['c_p'] = kmeans.cluster_centers_[kmeans.labels_,0]
tabela['c_v'] = kmeans.cluster_centers_[kmeans.labels_,1]
tabela['c_e'] = kmeans.cluster_centers_[kmeans.labels_,2]
tabela['c_d'] = kmeans.cluster_centers_[kmeans.labels_,3]
tabela['c_sg'] = kmeans.cluster_centers_[kmeans.labels_,4]
tabela['c_slope'] = kmeans.cluster_centers_[kmeans.labels_,5]
tabela_sort = tabela.sort_values(by=['2-P','4-V','9-SG'], ascending=False)
tabela_sort.style.background_gradient(cmap='Blues')


c:\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,1-Time,3-J,4-V,5-E,6-D,7-GP,8-GC,2-P,9-SG,cluster,cluster_pred,cl_o,cl_1,cl_2,cl_3,cl_4,time,slope,c_p,c_v,c_e,c_d,c_sg,c_slope
3,BOT,28,18,5,5,45.000000,18.000000,59,27.000000,4,4,0.000000,0.231500,0.238700,0.001100,0.528700,BOT,2.080000,59.000000,18.000000,5.000000,5.000000,27.000000,2.080000
4,RED,28,14,10,4,40.000000,24.000000,52,16.000000,2,2,0.000100,0.129400,0.804300,0.028700,0.037500,RED,1.830000,49.666667,13.666667,8.666667,5.666667,14.666667,1.746667
8,FLA,28,14,8,6,41.000000,31.000000,50,10.000000,2,2,0.001000,0.380800,0.482000,0.052200,0.084000,FLA,1.770000,49.666667,13.666667,8.666667,5.666667,14.666667,1.746667
0,PAL,28,13,8,7,42.000000,24.000000,47,18.000000,2,2,0.001100,0.416500,0.420700,0.085800,0.075900,PAL,1.640000,49.666667,13.666667,8.666667,5.666667,14.666667,1.746667
2,CAP,28,12,9,7,41.000000,32.000000,45,9.000000,1,2,0.003100,0.356000,0.409100,0.200800,0.031000,CAP,1.600000,42.333333,11.833333,6.833333,9.333333,5.500000,1.520000
9,GRE,28,13,5,10,43.000000,40.000000,44,3.000000,1,1,0.020100,0.758900,0.061700,0.084500,0.074900,GRE,1.700000,42.333333,11.833333,6.833333,9.333333,5.500000,1.520000
6,CAM,28,12,7,9,32.000000,22.000000,43,10.000000,1,1,0.009100,0.595400,0.162500,0.182800,0.050200,CAM,1.500000,42.333333,11.833333,6.833333,9.333333,5.500000,1.520000
5,FOR,28,12,6,10,33.000000,29.000000,42,4.000000,1,1,0.024500,0.679200,0.078300,0.173000,0.045100,FOR,1.480000,42.333333,11.833333,6.833333,9.333333,5.500000,1.520000
10,FLU,28,12,6,10,37.000000,35.000000,42,2.000000,1,1,0.029100,0.676000,0.074600,0.178500,0.041900,FLU,1.540000,42.333333,11.833333,6.833333,9.333333,5.500000,1.520000
12,SAO,28,10,8,10,34.000000,29.000000,38,5.000000,1,3,0.027200,0.372300,0.084500,0.505200,0.010800,SAO,1.300000,42.333333,11.833333,6.833333,9.333333,5.500000,1.520000


In [42]:
pontuacao_2 = pt_pontuacao_cum[pt_pontuacao_cum['rodada'] > 19]
pontuacao_2

time,rodada,AME,BAH,BOT,CAM,CAP,CFC,COR,CRU,CUI,...,FLU,FOR,GOI,GRE,INT,PAL,RED,SAN,SAO,VAS
19,20,10,21,48,27,32,14,25,25,28,...,34,29,23,34,24,37,32,21,28,19
20,21,13,21,51,30,33,14,26,25,28,...,35,32,24,37,25,40,35,21,28,19
21,22,16,22,51,31,34,14,27,26,28,...,38,32,25,40,26,41,36,21,31,20
22,23,17,25,51,34,37,14,27,29,29,...,38,35,25,40,29,44,39,21,31,23
23,24,17,25,51,37,40,14,30,29,29,...,41,38,26,43,29,44,42,24,31,26
24,25,18,25,52,40,40,17,30,30,32,...,41,39,27,44,29,44,45,27,34,26
25,26,18,28,55,40,41,20,31,31,33,...,41,42,27,44,32,44,46,30,35,27
26,27,18,31,58,43,44,20,32,31,36,...,42,42,30,44,32,44,49,30,35,30
27,28,19,34,59,43,45,20,33,34,37,...,42,42,31,44,35,47,52,30,38,30


In [43]:
colunas = pontuacao_2.columns
df_regressao = pd.DataFrame()
colunas = pontuacao_2.columns
regressao_data = []

for coluna in colunas:
    if coluna != 'rodada':
        X = pontuacao_2['rodada'].values.reshape(-1, 1)
        y = pontuacao_2[coluna].values.reshape(-1, 1)
        regressor = LinearRegression()
        regressor.fit(X, y)
        A = regressor.intercept_[0]
        B = regressor.coef_[0][0]
        x = A + (B * 38)
        regressao_data.append({
            'time': coluna,
            'intercept': round(A, 2),
            'slope': round(B, 2),
            'pontuacao_final': round(x, 2)
        })

df_regressao = pd.DataFrame(regressao_data)
df_regressao.sort_values(by='pontuacao_final', ascending=False)

,time,intercept,slope,pontuacao_final
16,RED,-17.42,2.47,76.31
2,BOT,23.29,1.23,70.16
9,FLA,-1.36,1.78,66.41
3,CAM,-14.69,2.12,65.74
4,CAP,-2.36,1.70,62.24
11,FOR,-5.62,1.77,61.51
13,GRE,11.91,1.22,58.14
15,PAL,19.58,0.97,56.31
10,FLU,14.31,1.03,53.58
18,SAO,3.53,1.20,49.13
